### The controller

In NVFlare, a `Controller` is a server-side component that manages the job execution and orchestration of tasks. Here, since we're running a P2P algorithm, we'll implement a custom controller whose main job is to load and broadcast the network configuration, and initiate/terminate the execution of a P2P distributed optimization algorithm. Let's call it `DistOptController`. As a subclass of `Controller`, it must implement 3 methods:

- `start_controller` which is called at the beginning of the run
- `control_flow` defining the main control flow of the controller (in this case, broadcasting the configuration and asking clients to run the algorithm)
- `stop_controller`, called at the end of the run

```python
from nvflare.apis.impl.controller import Controller

class DistOptController(Controller):

    def control_flow(self, abort_signal: Signal, fl_ctx: FLContext):
        # Broadcast configuration to clients
        ...

        # Run the algorithm
        ...

    def start_controller(self, fl_ctx: FLContext):
        pass

    def stop_controller(self, fl_ctx: FLContext):
        pass
```

We won't do anything fancy during the start and stop phase, so let's focus on the `control_flow` and implement the two steps. To do so, we first need to override the `__init__` method to take a `Config` object as an argument. 

```python
from nvflare.app_opt.p2p.types import Config

class DistOptController(Controller):
    def __init__(
        self,
        config: Config,
        *args,
        **kwargs,
    ):
        super().__init__(*args, **kwargs)
        self.config = config

    ... 
```

Now, in the `control_flow` method we can send the local configurations to each client and, once they receive them, ask them to run the algorithm. We'll do so by sending `Task`s to each client. In NVFlare, a `Task` is a piece of work that is assigned by the Controller to client workers. Depending on how the task is assigned (broadcast, send, or relay), the task will be performed by one or more clients.

In fact, on one hand, we'll use the `send_and_wait` method to send the `"config"` task to each client, since each client will potentially have a different config (because of different neighbors); on the other hand, to run the algorith, we'll use the `broadcast_and_wait`, which broadcasts the same `"run_algorithm"` task to all clients and waits for all clients to respond/complete the task. As you see, each task specifies a `name` - in this case, `"config"` and `"run_algorithm"` - let's remember those as they'll need to be the same in the control flow of each client.


```python
from nvflare.apis.controller_spec import Task
from nvflare.apis.dxo import DXO, DataKind
from nvflare.apis.fl_context import FLContext
from nvflare.apis.signal import Signal

class DistOptController(Controller):

    ...

    def control_flow(self, abort_signal: Signal, fl_ctx: FLContext):
        # Send network config (aka neighors info) to each client
        for node in self.config.network.nodes:
            task = Task(
                name="config",
                data=DXO(
                    data_kind=DataKind.APP_DEFINED,
                    data={"neighbors": [n.__dict__ for n in node.neighbors]},
                ).to_shareable(),
            )
            self.send_and_wait(task=task, targets=[node.id], fl_ctx=fl_ctx)

        # Run algorithm (with extra params if any passed as data)
        targets = [node.id for node in self.config.network.nodes]
        self.broadcast_and_wait(
            task=Task(
                name="run_algorithm",
                data=DXO(
                    data_kind=DataKind.APP_DEFINED,
                    data={key: value for key, value in self.config.extra.items()},
                ).to_shareable(),
            ),
            targets=targets,
            min_responses=0,
            fl_ctx=fl_ctx,
        )
    
    ... 
```

And that's it, our `DistOptController` is ready. The complete implementation of the `DistOptController` can be found in `nvflare/app_opt/p2p/controllers/dist_opt_controller.py`.
